---
layout: post
title: altair widget test
date: 2026-02-02
author: Yair Mau
toc: true
format:
  html:
    code-fold: true
    code-summary: "Show the code"
categories: [math, physics]
---

In [1]:
#| code-summary: "import libraries"
#| echo: false
import numpy as np
import pandas as pd
import altair as alt

In [ ]:
import numpy as np
import pandas as pd
import altair as alt

# disable altair's row limit (safe for browser-rendered charts)
alt.data_transformers.disable_max_rows()

blue = "#0343df"
red  = "#e50000"

x = np.linspace(0, 3, 301)
df_x = pd.DataFrame({"x": x})

# shared slider
x0 = alt.param(
    name="x0",
    value=2.5,
    bind=alt.binding_range(min=0, max=3, step=0.1, name="x₀")
)

# shared x/y encodings (apply to EVERY layer)
enc = {
    "x": alt.X("x:Q", scale=alt.Scale(domain=[0, 3]), axis=alt.Axis(title="x →")),
    "y": alt.Y("y:Q", scale=alt.Scale(domain=[-8, 8]), axis=alt.Axis(title="↑ y"))
}

# a base chart used only for convenience (data + params + size)
base = (
    alt.Chart(df_x)
    .add_params(x0)
    .properties(width=600, height=400)
)

# parabola y = x^2
parabola = (
    base.transform_calculate(y="datum.x * datum.x")
    .mark_line(stroke="black", strokeWidth=4)
    .encode(**enc)
)

# tangent line y = p*x - g
tangent = (
    base.transform_calculate(
        p1="2 * x0",
        g="(2 * x0) * x0 - (x0 * x0)",
        y="datum.x * datum.p1 - datum.g"
    )
    .mark_line(stroke=blue, strokeWidth=5)
    .encode(**enc)
)

# helper: a 1-row chart that still shares params/size
one = alt.Chart(pd.DataFrame([{}])).add_params(x0).properties(width=600, height=400)

# y=0 rule (use shared y scale by giving it x field too)
zero_rule = (
    alt.Chart(pd.DataFrame({"x": [0, 3], "y": [0, 0]}))
    .properties(width=600, height=400)
    .mark_line(stroke="#aaa", strokeWidth=2, opacity=0.5)
    .encode(**enc)
)

red_vertical = (
    base
    .transform_filter("datum.x == 0")
    .transform_calculate(
        p1="2 * x0",
        g="(2 * x0) * x0 - (x0 * x0)",
        x="0",
        y="0",
        y2="-datum.g"
    )
    .mark_rule(stroke=red, strokeWidth=5)
    .encode(
        x=enc["x"],
        y=enc["y"],
        y2=alt.Y2("y2:Q")
    )
)

# # red vertical segment at x=0 from y=0 to y=-g
# red_vertical = (
#     one.transform_calculate(
#         p1="2 * x0",
#         g="(2 * x0) * x0 - (x0 * x0)",
#         x="0",
#         y="0",
#         y2="-datum.g"
#     )
#     .mark_rule(stroke=red, strokeWidth=5)
#     .encode(
#         x=enc["x"],
#         y=enc["y"],
#         y2=alt.Y2("y2:Q")
#     )
# )

dashed_x0 = (
    base
    .transform_filter("datum.x == 0")
    .transform_calculate(
        x="x0",
        y="0",
        y2="x0 * x0"
    )
    .mark_rule(stroke="black", strokeWidth=2, strokeDash=[4, 4])
    .encode(
        x=enc["x"],
        y=enc["y"],
        y2=alt.Y2("y2:Q")
    )
)

# # dashed vertical at x=x0 from y=0 to y=x0^2
# dashed_x0 = (
#     one.transform_calculate(
#         x="x0",
#         y="0",
#         y2="x0 * x0"
#     )
#     .mark_rule(stroke="black", strokeWidth=2, strokeDash=[4, 4])
#     .encode(
#         x=enc["x"],
#         y=enc["y"],
#         y2=alt.Y2("y2:Q")
#     )
# )

# # point at (x0, x0^2)
# point = (
#     one.transform_calculate(x="x0", y="x0 * x0")
#     .mark_point(filled=True, size=200, color=red, stroke="white", strokeWidth=2)
#     .encode(**enc)
# )

# point at (x0, x0^2) -- robust version (uses df_x, filtered to one row)
point = (
    base
    .transform_filter("datum.x == 0")          # keep one row, but same dataset
    .transform_calculate(
        x="x0",
        y="x0 * x0"
    )
    .mark_point(filled=True, size=200, color=red, stroke="white", strokeWidth=2)
    .encode(**enc)
)

# label f(x)
label_fx = (
    alt.Chart(pd.DataFrame([{"x": 2.7, "y": 8.0, "text": "f(x)"}]))
    .properties(width=600, height=400)
    .mark_text(fontSize=20, fontWeight="bold", dy=-10)
    .encode(x=enc["x"], y=enc["y"], text="text:N")
)

# label g at (0, -g/2)
label_g = (
    one.transform_calculate(
        p1="2 * x0",
        g="(2 * x0) * x0 - (x0 * x0)",
        x="0",
        y="-datum.g / 2",
        text="'g'"
    )
    .mark_text(fontSize=20, fontWeight="bold", dx=15, color=red)
    .encode(x=enc["x"], y=enc["y"], text="text:N")
)

# label x₀ at (x0, 0)
label_x0 = (
    one.transform_calculate(x="x0", y="0", text="'x₀'")
    .mark_text(fontSize=20, fontWeight="bold", dy=12, color="black")
    .encode(x=enc["x"], y=enc["y"], text="text:N")
)

# label tangent text rotated
label_tangent = (
    one.transform_calculate(
        p1="2 * x0",
        g="(2 * x0) * x0 - (x0 * x0)",
        angle="-atan(datum.p1*(3/16)*(400/600)) * (180 / PI)",
        x="2",
        y="2 * datum.p1 - datum.g - 0.5",
        text="'tangent line y = px - g'"
    )
    .mark_text(fontSize=20, fontWeight="bold", dy=12, color=blue)
    .encode(x=enc["x"], y=enc["y"], text="text:N", angle="angle:Q")
)

chart = (
    alt.layer(
        zero_rule,
        parabola,
        tangent,
        red_vertical,
        dashed_x0,
        point,
        label_fx,
        label_g,
        label_x0,
        label_tangent
    )
    .properties(width=600, height=400)
    .configure_axis(grid=True, titleFontSize=20, titleFontWeight="bold", labelFontSize=14)
)

chart


alt.LayerChart(...)